<a href="https://colab.research.google.com/github/currencyfxjle/KNAPSACK-_GDF-_7.0/blob/main/KNAPSACK__GDF__7_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread
!pip install gspread-dataframe
!pip install --upgrade oauth2client
!pip install xlsxwriter

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

# List files and folders under My Drive
base_path = '/content/drive/My Drive'
for root, dirs, files in os.walk(base_path):
    print(f"Directory: {root}")
    for directory in dirs:
        print(f"  Sub-directory: {directory}")
    for file in files:
        print(f"  File: {file}")

import os

correct_path = '/content/drive/My Drive/Colab Notebooks/GDF - Knapsack/2025 Project Agenda.gsheet'

if os.path.exists(correct_path):
    print("File exists:", correct_path)
else:
    print("File does NOT exist:", correct_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.4 MB/s eta 0:00:00
Mounted at /content/drive
Directory: /content/drive/My Drive
  Sub-directory: Risk Modeling
  Sub-directory: Colab Notebooks
  File: Financial Concepts   Solar Energy Greenday Finance.gdoc
  File: config.zip
  File: 2024 Project Allocation Results (7).gsheet
  File: 2024 Project Allocation Results (6).gsheet
  File: 2024 Project Allocation Results (5).gsheet
  File: 2024 Project Allocation Results (4).gsheet
  File: 2024 Project Allocation Results (3).gsheet
  File: 2024 Project Allocation Results (2).gsheet
  File: 2024 Project Allocation Results (1).gsheet
  File: 2024 Project Allocation Results.gsheet
  File: Estados de Resultados - Victor Lizarraga.gdoc
  File: Ajustes  knapsack Lalo.gsheet
  File: Backup 2025.gsheet
  File: Ruta calculadora.gdoc
  File: KNAPSACK - GDF - Last Version 2.1.ipynb
  File: Knapsack Project.gdoc
Directory: /content/drive/My Drive/Risk Modeling
  File: Risk Model - Volatility 

In [7]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Define Priorities (Customizable)
PRIORITY = {
    'demand_state': {'CA': 1, 'Other': 4},  # CA has higher priority (1), others have lower priority (4)
    'low_appetite': 4,  # Medium priority for low appetite
    'early_dates': 2,   # Lower priority for early dates
    'profitability': 3  # Lowest priority for profitability
}

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']  # Only consider projects with 'Y'
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['Poject_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')

    # Clean and convert GDF Profit (remove '%' and convert to numeric)
    supply_df['GDF Profit'] = supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100  # Convert percentage to decimal

    supply_df = supply_df.dropna()

    # Sort projects by Poject_date (ascending) and GDF Profit (descending)
    supply_df = supply_df.sort_values(
        by=['Poject_date', 'GDF Profit'],
        ascending=[PRIORITY['early_dates'] == 3, PRIORITY['profitability'] == 4]
    )

    # Save original Est Cash Amount and add a "Fully Allocated" flag
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Categorize Est Cash Amount into buckets
    supply_df['Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )
    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['historic allocated amount'] = pd.to_numeric(demand_df['allocated_amount'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['original_pending_amount'] = demand_df['updated pending amount']  # Save original pending amount
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df = demand_df.dropna()

    # Add a column to track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Categorize Total Appetite into buckets
    demand_df['Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add state priority for demand (CA has higher priority)
    demand_df['Demand State Priority'] = demand_df['State'].apply(
        lambda x: PRIORITY['demand_state']['CA'] if x == 'CA' else PRIORITY['demand_state']['Other']
    )
    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    total_allocated = 0
    allocations = []
    unallocated_demand = []
    project_tracking = []

    # Sort purchasers by state priority (CA first) and then by approximated_appetite (ascending for low appetite first)
    demand_df = demand_df.sort_values(
        by=['Demand State Priority', 'approximated_appetite'],
        ascending=[True, PRIORITY['low_appetite'] == 2]
    )

    # Sort projects by Poject_date (ascending) and GDF Profit (descending)
    supply_df = supply_df.sort_values(
        by=['Poject_date', 'GDF Profit'],
        ascending=[PRIORITY['early_dates'] == 3, PRIORITY['profitability'] == 4]
    )

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['updated pending amount']
        remaining_appetite = purchaser['remaining_appetite']  # Track remaining appetite
        effective_date = purchaser['effective_date_purchaser']

        # Iterate through projects (sorted by date and profit)
        for _, project in supply_df.iterrows():
            if project['Fully Allocated']:
                continue  # Skip already allocated projects

            project_cost = project['Est Cash Amount']
            project_name = project['Proyect Name']
            project_id = project['Project ID']
            project_date = project['Poject_date']

            # Check if the project can be fully acquired and respects the date constraint
            if (project_cost <= remaining_pending_amount and
                project_cost <= remaining_appetite and  # Ensure it fits within remaining appetite
                project_date >= effective_date):
                # Allocate the project
                allocations.append({
                    "Purchaser": purchaser['purchaser_name'],
                    "Project ID": project_id,
                    "Project Name": project_name,
                    "Original Est Cash Amount": project['Original Est Cash Amount'],
                    "Allocated Amount": project_cost,
                    "Total Appetite": purchaser['approximated_appetite'],  # Original total appetite
                    "Remaining Appetite": remaining_appetite - project_cost,  # Updated remaining appetite
                    "Updated Pending Amount": remaining_pending_amount - project_cost,
                    "Effective Date Purchaser": effective_date,
                    "Project Date": project_date.date(),
                    "Fully Allocated": "Yes",
                    "Purchaser Category": purchaser['Category'],
                    "Project Category": project['Category'],
                    "GDF Profit": project['GDF Profit'],
                    "Demand State": purchaser['State'],  # Include demand state in allocations
                    "Supply State": project['State']  # Include supply state in allocations
                })
                total_allocated += project_cost
                remaining_pending_amount -= project_cost
                remaining_appetite -= project_cost  # Reduce remaining appetite
                supply_df.loc[project.name, 'Fully Allocated'] = True  # Mark project as fully allocated

                # Track project allocation
                project_tracking.append({
                    "Project ID": project_id,  # Include Project ID
                    "Project Name": project_name,
                    "Total Chunks": 1,  # Assuming each project is a single chunk
                    "Chunks Acquired": 1,
                    "Chunks Remaining": 0,
                    "Acquired Amounts": project_cost,
                    "Acquired By": purchaser['purchaser_name'],
                    "Total Appetite": purchaser['approximated_appetite'],  # Original total appetite
                    "Updated Pending Amount": remaining_pending_amount,
                    "Project Category": project['Category'],
                    "GDF Profit": project['GDF Profit'],
                    "Demand State": purchaser['State'],  # Include demand state in project tracking
                    "Supply State": project['State']  # Include supply state in project tracking
                })

                if remaining_pending_amount <= 0 or remaining_appetite <= 0:
                    break  # Move to the next purchaser

        # Update the purchaser's remaining pending amount and remaining appetite
        demand_df.at[idx, 'updated pending amount'] = remaining_pending_amount
        demand_df.at[idx, 'remaining_appetite'] = remaining_appetite  # Track remaining appetite

        # Track unallocated demand with specified columns
        if remaining_pending_amount > 0 or remaining_appetite > 0:
            unallocated_demand.append({
                "Purchaser Name": purchaser['purchaser_name'],
                "Approximated Appetite": purchaser['approximated_appetite'],
                "Original Pending Amount": purchaser['original_pending_amount'],
                "Allocated Amount": purchaser['approximated_appetite'] - remaining_appetite,
                "Updated Pending Amount": remaining_pending_amount,
                "Remaining Appetite": remaining_appetite,
                "Deal Stage": purchaser['deal_stage'],
                "Demand State": purchaser['State'],
                "Effective Date Purchaser": purchaser['effective_date_purchaser'],
                "Category": purchaser['Category'],
                "Demand State Priority": purchaser['Demand State Priority']
            })

    return total_allocated, allocations, unallocated_demand, supply_df, project_tracking

# Iterative Allocation Process
def iterative_allocation(supply_df, demand_df, max_iterations=5):
    iteration = 0
    total_allocated = 0
    all_allocations = []
    all_unallocated_demand = []
    project_tracking = []

    while iteration < max_iterations:
        iteration += 1
        current_allocated, allocations, unallocated_demand, remaining_supply, tracking = match_supply_to_demand(supply_df, demand_df)
        total_allocated += current_allocated
        all_allocations.extend(allocations)
        all_unallocated_demand.extend(unallocated_demand)
        project_tracking.extend(tracking)

        if current_allocated == 0:
            break

        supply_df = remaining_supply[remaining_supply['Est Cash Amount'] > 0].copy()

    return total_allocated, all_allocations, all_unallocated_demand, remaining_supply, demand_df, project_tracking

# Save Results to Separate Tabs in Excel
def save_to_excel(allocations, summary, unallocated_demand, remaining_supply, project_tracking, filename):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
        pd.DataFrame(summary).to_excel(writer, sheet_name="Summary", index=False)
        pd.DataFrame(unallocated_demand).to_excel(writer, sheet_name="Unallocated Demand", index=False)
        remaining_supply.to_excel(writer, sheet_name="Remaining Supply", index=False)
        pd.DataFrame(project_tracking).to_excel(writer, sheet_name="Project Tracking", index=False)  # Keep detailed rows

    print(f"Results saved to: {filename}")

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()
total_allocated, allocations, unallocated_demand, remaining_supply, remaining_demand, project_tracking = iterative_allocation(supply_df, demand_df)

# Calculations for Summary
total_project_value = supply_df['Original Est Cash Amount'].sum()

# Calculate Remaining Supply Value After: Sum of Est Cash Amount for projects with Fully Allocated = False
remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

# Original Total Unallocated Value (Total Appetite)
total_unallocated_value_appetite = demand_df['approximated_appetite'].sum()

# Total Allocated: Sum of Allocated Amount in allocations
total_allocated = sum(project['Allocated Amount'] for project in allocations)

# Total Pending Amount: Original Total Unallocated Value (Total Appetite) minus Total Allocated
total_pending_amount = total_unallocated_value_appetite - total_allocated

# Validation: Ensure Acquired Amounts match Allocated Amounts
total_acquired_amounts = sum(project['Acquired Amounts'] for project in project_tracking)
if total_allocated == total_acquired_amounts:
    print("Validation Passed: Acquired Amounts match Allocated Amounts.")
else:
    print(f"Validation Failed: Acquired Amounts ({total_acquired_amounts}) do not match Allocated Amounts ({total_allocated}).")

# Summary Data
summary_data = pd.DataFrame([
    {"Metric": "Original Total Unallocated Value (Total Appetite)", "Value": total_unallocated_value_appetite},
    {"Metric": "Original Total Project Value (Est Cash Amount)", "Value": total_project_value},
    {"Metric": "Total Allocated", "Value": total_allocated},
    {"Metric": "Remaining Supply Value After", "Value": remaining_supply_value},
    {"Metric": "Total Pending Amount", "Value": total_pending_amount},
])

# Save Results to Excel with Tabs
save_to_excel(
    allocations,
    summary_data,
    unallocated_demand,
    remaining_supply,
    project_tracking,
    "allocation_results.xlsx"
)

# Results Summary
print("\nResults Summary:")
print(f"Original Total Unallocated Value (Total Appetite): {total_unallocated_value_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {total_project_value}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")

Validation Passed: Acquired Amounts match Allocated Amounts.
Results saved to: allocation_results.xlsx

Results Summary:
Original Total Unallocated Value (Total Appetite): 110880429.1
Original Total Project Value (Est Cash Amount): 19123134.0
Total Allocated: 19123134.0
Remaining Supply Value After: 0.0
Total Pending Amount: 91757295.1


# **DASHBOARD ADAPTATIONS**

In [16]:
!pip install streamlit
!pip install pyngrok

In [17]:
!streamlit run dashboard.py --server.port 8501 &>/content/logs.txt &

In [18]:
%%writefile dashboard.py
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import streamlit as st  # Import Streamlit
import plotly.express as px  # Import Plotly Express for visualizations

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Define Priorities (Customizable)
PRIORITY = {
    'demand_state': {'CA': 1, 'Other': 4},  # CA has higher priority (1), others have lower priority (4)
    'low_appetite': 4,  # Medium priority for low appetite
    'early_dates': 2,   # Lower priority for early dates
    'profitability': 3  # Lowest priority for profitability
}

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']  # Only consider projects with 'Y'
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['Poject_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')

    # Clean and convert GDF Profit (remove '%' and convert to numeric)
    supply_df['GDF Profit'] = supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100  # Convert percentage to decimal

    supply_df = supply_df.dropna()

    # Sort projects by Poject_date (ascending) and GDF Profit (descending)
    supply_df = supply_df.sort_values(
        by=['Poject_date', 'GDF Profit'],
        ascending=[PRIORITY['early_dates'] == 3, PRIORITY['profitability'] == 4]
    )

    # Save original Est Cash Amount and add a "Fully Allocated" flag
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Categorize Est Cash Amount into buckets
    supply_df['Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )
    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['historic allocated amount'] = pd.to_numeric(demand_df['allocated_amount'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['original_pending_amount'] = demand_df['updated pending amount']  # Save original pending amount
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df = demand_df.dropna()

    # Add a column to track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Categorize Total Appetite into buckets
    demand_df['Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add state priority for demand (CA has higher priority)
    demand_df['Demand State Priority'] = demand_df['State'].apply(
        lambda x: PRIORITY['demand_state']['CA'] if x == 'CA' else PRIORITY['demand_state']['Other']
    )
    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    total_allocated = 0
    allocations = []
    unallocated_demand = []
    project_tracking = []

    # Sort purchasers by state priority (CA first) and then by approximated_appetite (ascending for low appetite first)
    demand_df = demand_df.sort_values(
        by=['Demand State Priority', 'approximated_appetite'],
        ascending=[True, PRIORITY['low_appetite'] == 2]
    )

    # Sort projects by Poject_date (ascending) and GDF Profit (descending)
    supply_df = supply_df.sort_values(
        by=['Poject_date', 'GDF Profit'],
        ascending=[PRIORITY['early_dates'] == 3, PRIORITY['profitability'] == 4]
    )

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['updated pending amount']
        remaining_appetite = purchaser['remaining_appetite']  # Track remaining appetite
        effective_date = purchaser['effective_date_purchaser']

        # Iterate through projects (sorted by date and profit)
        for _, project in supply_df.iterrows():
            if project['Fully Allocated']:
                continue  # Skip already allocated projects

            project_cost = project['Est Cash Amount']
            project_name = project['Proyect Name']
            project_id = project['Project ID']
            project_date = project['Poject_date']

            # Check if the project can be fully acquired and respects the date constraint
            if (project_cost <= remaining_pending_amount and
                project_cost <= remaining_appetite and  # Ensure it fits within remaining appetite
                project_date >= effective_date):
                # Allocate the project
                allocations.append({
                    "Purchaser": purchaser['purchaser_name'],
                    "Project ID": project_id,
                    "Project Name": project_name,
                    "Original Est Cash Amount": project['Original Est Cash Amount'],
                    "Allocated Amount": project_cost,
                    "Total Appetite": purchaser['approximated_appetite'],  # Original total appetite
                    "Remaining Appetite": remaining_appetite - project_cost,  # Updated remaining appetite
                    "Updated Pending Amount": remaining_pending_amount - project_cost,
                    "Effective Date Purchaser": effective_date,
                    "Project Date": project_date.date(),
                    "Fully Allocated": "Yes",
                    "Purchaser Category": purchaser['Category'],
                    "Project Category": project['Category'],
                    "GDF Profit": project['GDF Profit'],
                    "Demand State": purchaser['State'],  # Include demand state in allocations
                    "Supply State": project['State']  # Include supply state in allocations
                })
                total_allocated += project_cost
                remaining_pending_amount -= project_cost
                remaining_appetite -= project_cost  # Reduce remaining appetite
                supply_df.loc[project.name, 'Fully Allocated'] = True  # Mark project as fully allocated

                # Track project allocation
                project_tracking.append({
                    "Project ID": project_id,  # Include Project ID
                    "Project Name": project_name,
                    "Total Chunks": 1,  # Assuming each project is a single chunk
                    "Chunks Acquired": 1,
                    "Chunks Remaining": 0,
                    "Acquired Amounts": project_cost,
                    "Acquired By": purchaser['purchaser_name'],
                    "Total Appetite": purchaser['approximated_appetite'],  # Original total appetite
                    "Updated Pending Amount": remaining_pending_amount,
                    "Project Category": project['Category'],
                    "GDF Profit": project['GDF Profit'],
                    "Demand State": purchaser['State'],  # Include demand state in project tracking
                    "Supply State": project['State']  # Include supply state in project tracking
                })

                if remaining_pending_amount <= 0 or remaining_appetite <= 0:
                    break  # Move to the next purchaser

        # Update the purchaser's remaining pending amount and remaining appetite
        demand_df.at[idx, 'updated pending amount'] = remaining_pending_amount
        demand_df.at[idx, 'remaining_appetite'] = remaining_appetite  # Track remaining appetite

        # Track unallocated demand with specified columns
        if remaining_pending_amount > 0 or remaining_appetite > 0:
            unallocated_demand.append({
                "Purchaser Name": purchaser['purchaser_name'],
                "Approximated Appetite": purchaser['approximated_appetite'],
                "Original Pending Amount": purchaser['original_pending_amount'],
                "Allocated Amount": purchaser['approximated_appetite'] - remaining_appetite,
                "Updated Pending Amount": remaining_pending_amount,
                "Remaining Appetite": remaining_appetite,
                "Deal Stage": purchaser['deal_stage'],
                "Demand State": purchaser['State'],
                "Effective Date Purchaser": purchaser['effective_date_purchaser'],
                "Category": purchaser['Category'],
                "Demand State Priority": purchaser['Demand State Priority']
            })

    return total_allocated, allocations, unallocated_demand, supply_df, project_tracking

# Iterative Allocation Process
def iterative_allocation(supply_df, demand_df, max_iterations=5):
    iteration = 0
    total_allocated = 0
    all_allocations = []
    all_unallocated_demand = []
    project_tracking = []

    while iteration < max_iterations:
        iteration += 1
        current_allocated, allocations, unallocated_demand, remaining_supply, tracking = match_supply_to_demand(supply_df, demand_df)
        total_allocated += current_allocated
        all_allocations.extend(allocations)
        all_unallocated_demand.extend(unallocated_demand)
        project_tracking.extend(tracking)

        if current_allocated == 0:
            break

        supply_df = remaining_supply[remaining_supply['Est Cash Amount'] > 0].copy()

    return total_allocated, all_allocations, all_unallocated_demand, remaining_supply, demand_df, project_tracking

# Save Results to Separate Tabs in Excel
def save_to_excel(allocations, summary, unallocated_demand, remaining_supply, project_tracking, filename):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
        pd.DataFrame(summary).to_excel(writer, sheet_name="Summary", index=False)
        pd.DataFrame(unallocated_demand).to_excel(writer, sheet_name="Unallocated Demand", index=False)
        remaining_supply.to_excel(writer, sheet_name="Remaining Supply", index=False)
        pd.DataFrame(project_tracking).to_excel(writer, sheet_name="Project Tracking", index=False)  # Keep detailed rows

    print(f"Results saved to: {filename}")

# Streamlit Dashboard
def main():
    st.title("Project Allocation Dashboard")

    # Sidebar for user inputs
    st.sidebar.header("Configuration")
    max_iterations = st.sidebar.slider("Max Iterations", 1, 10, 5)

    # Fetch data
    supply_df = fetch_supply()
    demand_df = fetch_demand()

    # Run allocation process
    total_allocated, allocations, unallocated_demand, remaining_supply, remaining_demand, project_tracking = iterative_allocation(supply_df, demand_df, max_iterations)

    # Display results
    st.header("Allocations")
    st.dataframe(pd.DataFrame(allocations))

    st.header("Unallocated Demand")
    st.dataframe(pd.DataFrame(unallocated_demand))

    st.header("Remaining Supply")
    st.dataframe(remaining_supply)

    st.header("Project Tracking")
    st.dataframe(pd.DataFrame(project_tracking))

    # Summary
    st.header("Summary")
    total_project_value = supply_df['Original Est Cash Amount'].sum()
    remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()
    total_unallocated_value_appetite = demand_df['approximated_appetite'].sum()
    total_allocated = sum(project['Allocated Amount'] for project in allocations)
    total_pending_amount = total_unallocated_value_appetite - total_allocated

    st.write(f"**Original Total Unallocated Value (Total Appetite):** {total_unallocated_value_appetite}")
    st.write(f"**Original Total Project Value (Est Cash Amount):** {total_project_value}")
    st.write(f"**Total Allocated:** {total_allocated}")
    st.write(f"**Remaining Supply Value After:** {remaining_supply_value}")
    st.write(f"**Total Pending Amount:** {total_pending_amount}")

    # Visualizations
    st.header("Visualizations")

    # Bar chart for total chunks per project
    project_summary = pd.DataFrame(project_tracking).groupby('Project Name').agg(
        Total_Chunks=('Total Chunks', 'sum'),
        Total_Value=('Acquired Amounts', 'sum')
    ).reset_index()
    fig = px.bar(project_summary, x='Project Name', y='Total_Chunks', title='Total Chunks per Project', text='Total_Chunks')
    fig.update_traces(texttemplate='%{text}', textposition='outside')
    st.plotly_chart(fig)

    # Bar chart for total value per project
    fig = px.bar(project_summary, x='Project Name', y='Total_Value', title='Total Value per Project', text='Total_Value')
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    st.plotly_chart(fig)

    # Heatmap for allocation matrix
    allocation_matrix = pd.DataFrame(allocations).pivot_table(index='Purchaser', columns='Project Category', values='Allocated Amount', aggfunc='sum').fillna(0)
    fig = px.imshow(allocation_matrix, title='Allocation Matrix (Purchaser vs Project Category)', labels=dict(x="Project Category", y="Purchaser", color="Allocated Amount"))
    st.plotly_chart(fig)

    # Box plot for GDF Profit distribution
    fig = px.box(supply_df, x='Category', y='GDF Profit', title='GDF Profit Distribution by Project Category')
    st.plotly_chart(fig)

    # Sunburst chart for hierarchical view of allocations
    allocations_df = pd.DataFrame(allocations)
    allocations_df['Parent'] = 'Allocations'
    allocations_df['Value'] = allocations_df['Allocated Amount']
    fig = px.sunburst(allocations_df, path=['Parent', 'Purchaser', 'Project Category'], values='Value', title='Hierarchical View of Allocations')
    st.plotly_chart(fig)

    # Histogram for project sizes
    fig = px.histogram(supply_df, x='Est Cash Amount', nbins=20, title='Distribution of Project Sizes')
    st.plotly_chart(fig)

    # Stacked bar chart for allocated vs unallocated by category
    category_data = pd.DataFrame({
        'Project Category': ['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+'],
        'Allocated': [100, 200, 150, 300, 250],  # Replace with actual data
        'Unallocated': [50, 100, 75, 150, 125]  # Replace with actual data
    })
    fig = px.bar(category_data, x='Project Category', y=['Allocated', 'Unallocated'], title='Allocated vs Unallocated by Category', barmode='stack', labels={'value': 'Amount', 'variable': 'Type'})
    st.plotly_chart(fig)

    # Scatter plot for GDF Profit vs Allocated Amount
    allocations_df['Point Size'] = allocations_df['Allocated Amount'] / 100000
    fig = px.scatter(allocations_df, x='GDF Profit', y='Allocated Amount', color='Project Category', size='Point Size', hover_name='Project Name', hover_data={'Purchaser': True, 'Project Date': True, 'Allocated Amount': ':.2f', 'GDF Profit': ':.2%'}, title='GDF Profit vs Allocated Amount by Project Category', labels={'GDF Profit': 'GDF Profit (%)', 'Allocated Amount': 'Allocated Amount ($)', 'Project Category': 'Project Category'}, template='plotly_white')
    fig.update_traces(line=dict(dash='dash', width=1), selector=dict(mode='lines'))
    st.plotly_chart(fig)

    # Line chart for cumulative project allocation over time
    allocations_df['Project Date'] = pd.to_datetime(allocations_df['Project Date'])
    allocations_df = allocations_df.sort_values('Project Date')
    allocations_df['Cumulative Allocated'] = allocations_df['Allocated Amount'].cumsum()
    fig = px.line(allocations_df, x='Project Date', y='Cumulative Allocated', title='Cumulative Project Allocation Over Time')
    st.plotly_chart(fig)

    # Pie chart for allocated projects by category
    allocated_by_category = pd.DataFrame(allocations).groupby('Project Category').size().reset_index(name='Count')
    fig = px.pie(allocated_by_category, values='Count', names='Project Category', title='Allocated Projects by Category')
    st.plotly_chart(fig)

    # Bar chart for allocation summary
    metrics = ['Total Allocated', 'Remaining Supply', 'Total Pending Amount']
    values = [total_allocated, remaining_supply_value, total_pending_amount]
    bar_data = pd.DataFrame({'Metrics': metrics, 'Values': values})
    fig = px.bar(bar_data, x='Metrics', y='Values', title='Allocation Summary', text='Values')
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    st.plotly_chart(fig)

# Run the dashboard
if __name__ == "__main__":
    main()

Overwriting dashboard.py


In [19]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken
ngrok.set_auth_token("2Hab34WiFCqnyF8m2gU825zSaex_6BTKDpiB7rK3ifATw16zu")

# Create a tunnel to the Streamlit app
public_url = ngrok.connect(addr=8501, proto="http")
print("Streamlit app is live at:", public_url)

Streamlit app is live at: NgrokTunnel: "https://f37b-35-197-76-247.ngrok-free.app" -> "http://localhost:8501"
